In [1]:
import os 
import json
import pprint as pp
import tweepy
import glob
import networkx as nx
import pandas as pd
from pyvis.network import Network

In [2]:
#Twitter API credentials
api_key = "9yCV33tYVEjfxBhOBXUceRNkg"
api_secret = "STfRtWE5Au4g9v5W2nj5GyqWdS6HkPGcVCvlXEe6z7jUM5ADtL"
access_token = "748153809076105216-mgo6jZxbaWbnDWoTZOkPnuu8V5gs1WO"
access_secret = "EZ3sIXGz0NsBftEdVXkluCqR5BUcJIz1jMiAE0jTN7Wz8"

In [3]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')

Authentication completed successfully!


# JSON

In [4]:
data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}


# FOLLOWERS e FOLLOWING

In [5]:
def findUsers(userlist,mode): #mode: 0 = followers, 1 = following
    quantity=20
    l2list = {}

    for user in userlist:
        print(f"Processing user {user}")

        found = []
        l2list[user]=[]

        if mode==0:
            usergroup=tweepy.Cursor(api.followers,screen_name = user ).items(quantity)
        else:
            usergroup=tweepy.Cursor(api.friends,screen_name = user ).items(quantity)

        for item in usergroup:

            json_data = item._json

            found_user = {}
            found_user["id"] = json_data["id"]
            found_user["name"] = json_data["name"]
            found_user["screen_name"] = json_data["screen_name"]
            found_user["location"] = json_data["location"]
            found_user["followers_count"] = json_data["followers_count"]
            found_user["friends_count"] = json_data["friends_count"]
            found_user["protected"] = json_data["protected"]


            if item.friends_count < 1000 and item.followers_count<1000 and json_data["protected"] == False and (found_user not in found):
                found.append(found_user)
                l2list[user].append(item.screen_name)
                print(item.screen_name)

            if len(found) >= 5:
                break


        if mode==0:
            serialize_json(data_folder+"/users", user+"_followers.json", found)
        else:
            serialize_json(data_folder+"/users", user+"_following.json", found)

    return l2list

l1_followers=findUsers(["mizzaro","Miccighel_"],0)
l1_following=findUsers(["mizzaro","Miccighel_"],1)

l2a_followers_mizzaro=findUsers(l1_followers["mizzaro"],0)
l2a_following_mizzaro=findUsers(l1_followers["mizzaro"],1)

l2a_followers_miccighel=findUsers(l1_followers["Miccighel_"],0)
l2a_following_miccighel=findUsers(l1_followers["Miccighel_"],1)

l2b_followers_mizzaro=findUsers(l1_following["mizzaro"],0)
l2b_following_mizzaro=findUsers(l1_following["mizzaro"],1)

l2b_followers_miccighel=findUsers(l1_following["Miccighel_"],0)
l2b_following_miccighel=findUsers(l1_following["Miccighel_"],1)

Processing user mizzaro
OlegZendel
LyndaSaidLhadj
acai2021
rodger_benham
ColoursRiot
Data serialized to path: data/users/mizzaro_followers.json
Processing user Miccighel_
Il_Bale_
Outsider289
mizzaro
EUROARTISTS
fra_bomba6
Data serialized to path: data/users/Miccighel__followers.json
Processing user mizzaro
MaxZancanaro
CIRES_AU
mccart_richard
Miccighel_
rmit_cidda
Data serialized to path: data/users/mizzaro_following.json
Processing user Miccighel_
fra_bomba6
Outsider289
BiancolinoElisa
ZuzanPink
eglu81
Data serialized to path: data/users/Miccighel__following.json
Processing user OlegZendel
bevan_koopman
Marwah_k
ivrik
Dars2022_Sympo
ameertawfik
Data serialized to path: data/users/OlegZendel_followers.json
Processing user LyndaSaidLhadj
libinfopro
Christo12656926
gael_dias
Nthakur20
AthmaniAmine
Data serialized to path: data/users/LyndaSaidLhadj_followers.json
Processing user acai2021
TilburgU_DCA
srj_krishna
gabriellearuta
lesleywilton
sencerdeniz
Data serialized to path: data/users/

Rate limit reached. Sleeping for: 881


Il_Bale_
Outsider289
mizzaro
EUROARTISTS
fra_bomba6
Data serialized to path: data/users/Miccighel__followers.json
Processing user rmit_cidda
sumitsidana17
nowshed_bd
Dr_Hai_Dong
MaksudaBilkis
bitcoinub
Data serialized to path: data/users/rmit_cidda_followers.json
Processing user MaxZancanaro
njn
EnricoBlanzieri
MTStella
uhopper
speckandtech
Data serialized to path: data/users/MaxZancanaro_following.json
Processing user CIRES_AU
crc_gk
linhlt262626
JasminC42298109
DimityMiller
Data serialized to path: data/users/CIRES_AU_following.json
Processing user mccart_richard
quantschmant
kelsiemvn
DrJJRich
mintandburnRMIT
piumie99
Data serialized to path: data/users/mccart_richard_following.json
Processing user Miccighel_
fra_bomba6
Outsider289
BiancolinoElisa
ZuzanPink
eglu81
Data serialized to path: data/users/Miccighel__following.json
Processing user rmit_cidda
informagus
danulaera
RosieJonesNLP
DrJeffreyC
IEEEPerCom
Data serialized to path: data/users/rmit_cidda_following.json
Processing use

In [8]:
fileList=glob.glob(data_folder+"/users/*.json")

totalData=[]

for fileName in fileList:
    with open(fileName,'r',encoding="UTF-8") as file:
        data=json.load(file)
        #totalData.append(data)
        for user in data:
            totalData.append(user)

serialize_json(data_folder+"/total", "total_users.json", totalData)

Data serialized to path: data/total/total_users.json


# Dettagli del profilo dei followers di primo e secondo livello

In [11]:
def userDetails():

    fileList=glob.glob(data_folder+"/users/*_followers.json")

    for file in fileList:
        with open(file,'r',encoding="UTF-8") as rawdata:
            data=json.load(rawdata)
            for user in data:
                raw_data = api.get_user(user)
                json_data = raw_data._json
                profile_data = {}
                profile_data["id"] = json_data["id"]
                profile_data["name"] = json_data["name"]
                profile_data["screen_name"] = json_data["screen_name"]
                profile_data["location"] = json_data["location"]
                profile_data["description"] = json_data["description"]
                profile_data["followers_count"] = json_data["followers_count"]
                profile_data["friends_count"] = json_data["friends_count"]
                serialize_json(data_folder+"/profiledetails", "1_"+profile_data["screen_name"]+"_profile_data.json", profile_data)
                pp.pprint(profile_data)

userDetails()

Data serialized to path: data/1_MaxZancanaro_profile_data.json
{'description': '',
 'followers_count': 211,
 'friends_count': 197,
 'id': 1239642882,
 'location': '',
 'name': 'Massimo Zancanaro',
 'screen_name': 'MaxZancanaro'}
Data serialized to path: data/1_CIRES_AU_profile_data.json
{'description': '',
 'followers_count': 37,
 'friends_count': 100,
 'id': 1425660262653120517,
 'location': '',
 'name': 'ARC Training Centre for Information Resilience',
 'screen_name': 'CIRES_AU'}
Data serialized to path: data/1_mccart_richard_profile_data.json
{'description': '',
 'followers_count': 52,
 'friends_count': 83,
 'id': 1020854242150453253,
 'location': '',
 'name': 'Richard McCart',
 'screen_name': 'mccart_richard'}
Data serialized to path: data/1_Miccighel__profile_data.json
{'description': 'University Life, Computer Science, Photography, Reading, '
                'Gaming, Comics. PhD Student at the University of Udine. '
                "That's me.",
 'followers_count': 330,
 'friends

# Ultimi 5 tweet di ciascun follower di primo e secondo livello

In [13]:
def getTweets():

    fileList=glob.glob(data_folder+"/users/*_followers.json")

    for file in fileList:
        with open(file,'r',encoding="UTF-8") as rawdata:
            data=json.load(rawdata)
    
            for user in data:

                tweets = []
                for item in tweepy.Cursor(
                        api.user_timeline,
                        screen_name=user,
                        tweet_mode="extended"
                    ).items(5):
                    raw_tweet = item._json
                    tweet = {}
                    tweet["id"]= raw_tweet["id"]
                    tweet["created_at"] = raw_tweet["created_at"]
                    tweet["full_text"] = raw_tweet["full_text"],
                    tweet["favorite_count"]= raw_tweet["favorite_count"]
                    tweets.append(tweet)
                serialize_json(data_folder+"/tweets", user+"_recent_tweets.json", tweets)
                pp.pprint(tweets)

getTweets()

Data serialized to path: data/MaxZancanaro_recent_tweets.json
[{'created_at': 'Wed Nov 03 17:29:03 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @diunito: #design #ethics #trust #Worldusabilityday '
                '#WUD2021 #wudsigchi2021 #ux #wud #uxdesign #userexperience '
                '#ai #usability #uxresearc…',),
  'id': 1455950161264394251},
 {'created_at': 'Tue Sep 14 18:47:54 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @i3_FBK: Another successful initiative @Families_Share 😄\n'
                'Co-production of childcare can really improve '
                '#WorkLifeBalance and wellbeing.',),
  'id': 1437850611899379713},
 {'created_at': 'Tue Mar 09 18:15:34 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @elme85: 📣@TonolliLinda, @emmenomade, Roberto Cibin, Amon '
                'Rapp, @MaxZancana, @vincenzodandrea and I are organizing a '
                'workshop @chitaly20…',),
  'id': 1369351166430228490},
 {'created_at': 'Sun Mar 07 09:

# Rete sociale

In [6]:
#Recupero tutte le amicizie di ciascun utente

def getAllUsers():
    found=[]

    file=open(data_folder+"/total/total_users.json",'r',encoding="UTF-8")
    userlist=json.load(file)


    for user in userlist:
        found=[]
        found_user = {}
        print(user)
        print(f"Processing user {user['screen_name']}")

        usergroup_followers=tweepy.Cursor(api.followers,screen_name = user["screen_name"] ).items()
        usergroup_following=tweepy.Cursor(api.friends,screen_name = user["screen_name"] ).items()

        for item in usergroup_followers:

            json_data = item._json

            found_user = {}
            found_user["target_id"] = json_data["id"]
            found_user["target_screen_name"] = json_data["screen_name"]
            found_user["source_id"] = user["id"]
            found.append(found_user)


        serialize_json(data_folder+"/friendships", user["screen_name"]+"_followers.json", found)

        found = []

        for item in usergroup_following:

            json_data = item._json

            found_user = {}
            found_user["target_id"] = json_data["id"]
            found_user["target_screen_name"] = json_data["screen_name"]
            found_user["source_id"] = user["id"]
            found.append(found_user)


        serialize_json(data_folder+"/friendships", user["screen_name"]+"_following.json", found)

getAllUsers()

{'id': 18932422, 'name': 'mizzaro', 'screen_name': 'mizzaro', 'location': '', 'followers_count': 162, 'friends_count': 358, 'protected': False}
Processing user mizzaro
Data serialized to path: data/friendships/mizzaro_followers.json


Rate limit reached. Sleeping for: 889


Data serialized to path: data/friendships/mizzaro_following.json
{'id': 1440461778236829703, 'name': 'Christina Gowlett', 'screen_name': 'CGowlett1', 'location': '', 'followers_count': 5, 'friends_count': 15, 'protected': False}
Processing user CGowlett1
Data serialized to path: data/friendships/CGowlett1_followers.json
Data serialized to path: data/friendships/CGowlett1_following.json
{'id': 1136467769199910912, 'name': 'lizzie2626', 'screen_name': 'linhlt262626', 'location': 'Brisbane, Queensland', 'followers_count': 15, 'friends_count': 63, 'protected': False}
Processing user linhlt262626
Data serialized to path: data/friendships/linhlt262626_followers.json
Data serialized to path: data/friendships/linhlt262626_following.json
{'id': 769507902897856512, 'name': 'Hassan Khosravi', 'screen_name': 'haskhosravi', 'location': 'Brisbane, Vancouver', 'followers_count': 482, 'friends_count': 592, 'protected': False}
Processing user haskhosravi


Rate limit reached. Sleeping for: 886


Data serialized to path: data/friendships/haskhosravi_followers.json


...

Data serialized to path: data/friendships/AiLeibniz_followers.json
Data serialized to path: data/friendships/AiLeibniz_following.json
{'id': 524234866, 'name': 'Mike Schaekermann', 'screen_name': 'HardyShakerman', 'location': 'Waterloo, Ontario', 'followers_count': 145, 'friends_count': 115, 'protected': False}
Processing user HardyShakerman
Data serialized to path: data/friendships/HardyShakerman_followers.json
Data serialized to path: data/friendships/HardyShakerman_following.json
{'id': 3223120420, 'name': 'Alex Williams', 'screen_name': 'acwio', 'location': 'Knoxville, Tennessee', 'followers_count': 933, 'friends_count': 730, 'protected': False}
Processing user acwio


Rate limit reached. Sleeping for: 882
Rate limit reached. Sleeping for: 889
Rate limit reached. Sleeping for: 887


Data serialized to path: data/friendships/acwio_followers.json


Rate limit reached. Sleeping for: 889
Rate limit reached. Sleeping for: 889


Data serialized to path: data/friendships/acwio_following.json
{'id': 1430793928521904129, 'name': 'uq-hcc', 'screen_name': 'hcc_uq', 'location': '', 'followers_count': 35, 'friends_count': 41, 'protected': False}
Processing user hcc_uq
Data serialized to path: data/friendships/hcc_uq_followers.json
Data serialized to path: data/friendships/hcc_uq_following.json


In [5]:
# Creazione dataframe dei Nodi
def create_node(follower_of_user):
    #lettura json dell'utente desiderato
    with open(data_folder+"/total/"+follower_of_user+".json", "r", encoding="UTF-8") as read_file:
        data = json.load(read_file)
    #aggiunta dei nodi
    followers_nodes=[]
    for result in data:
        followers_nodes.append(result)
    #df_nodi_grafo
    df_nodes = pd.DataFrame(followers_nodes)
    return df_nodes

#print(create_node('total_users'))
df_nodes=create_node('total_users')

In [8]:
graph = nx.DiGraph(team="Elia Gazzola, Martina Moro, Cristiano Roman, Pierre Agnoletti")

for index,row in df_nodes.iterrows():
    graph.add_node(row["id"],
    id = row["id"],
    title = row["name"],
    color ="#ffff00",
    physics = False,
    name = row['name'],
    screen_name = row['screen_name'],
    location=row['location'],
    followers_count = row["followers_count"],
    following_count = row["friends_count"]
    #tweets
    )

def create_edges():
    with open(data_folder+"/total/total_users.json","r",encoding="UTF-8") as totalFile:
        data=json.load(totalFile)
        for user in data:
            with open(data_folder+"/friendships/"+user["screen_name"]+"_followers.json","r",encoding="UTF-8") as followers:
                data_fws=json.load(followers)
                for follower in data_fws:
                    print(user["screen_name"]+" --> "+follower["target_screen_name"])
                    graph.add_edge(user["id"],follower["target_id"])

            with open(data_folder+"/friendships/"+user["screen_name"]+"_following.json","r",encoding="UTF-8") as followings:
                data_fwing=json.load(followings)
                for following in data_fwing:
                    print(following["target_screen_name"]+" --> "+user["screen_name"])
                    graph.add_edge(following["target_id"],user["id"])

create_edges()

mizzaro --> OlegZendel
mizzaro --> LyndaSaidLhadj
mizzaro --> davideceolin
mizzaro --> acai2021
mizzaro --> CIKM2021
mizzaro --> rodger_benham
mizzaro --> ColoursRiot
mizzaro --> ChriShot90
mizzaro --> dungdx34
mizzaro --> flaviomartins
mizzaro --> WWRob
mizzaro --> JCAlbornozC
mizzaro --> FTHopf
mizzaro --> lgr3d
mizzaro --> antonio_mallia
mizzaro --> NewsIR_Workshop
mizzaro --> alb_purpura
mizzaro --> sculpepper
mizzaro --> ntcirstc
mizzaro --> acm_ictir
mizzaro --> emranrian55
mizzaro --> ida_mele
mizzaro --> EvALL_TEAM
mizzaro --> heartedlisa30
mizzaro --> ABH878
mizzaro --> essir2017
mizzaro --> fmnardini
mizzaro --> zuacsea
mizzaro --> JosianeMotheFr
mizzaro --> RankSys
mizzaro --> ACMSIGIR
mizzaro --> agchifu
mizzaro --> egaussier
mizzaro --> airs2016
mizzaro --> KevinRoitero
mizzaro --> abrightmoore
mizzaro --> dyaaa
mizzaro --> AleVergendo
mizzaro --> HamedZamani
mizzaro --> basilepp
mizzaro --> trecopensearch
mizzaro --> ictir16
mizzaro --> sarvk
mizzaro --> jarana_m
mizzaro 

In [9]:
def disegna_grafo(grafo):
    nt = Network(
        height ="100%",
        width = "100%",
        bgcolor="#222222",
        font_color="white",
        heading= grafo,
        directed=True
    )
    nt.from_nx(grafo)
    nt.barnes_hut()
    # nt.set_edge_smooth("continuous") #cambia formato di visualizzazione degli archi
    neighbor_map = nt.get_adj_list()
    for node in nt.nodes:
        node["value"] = len(neighbor_map[node["id"]])
    # print(str(node["id"])+":"+ str(node["value"]))
    nt.show("grafo.html")

disegna_grafo(graph)